In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np
from sklearn.preprocessing import StandardScaler


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
data = pd.read_csv('facies_vectors.csv')
data = data.fillna(data['PE'].mean())
feature_names = ['GR_diff_up', 'ILD_log10_diff_up', 'DeltaPHI_diff_up', 'PHIND_diff_up', 'PE_diff_up', 'NM_M_diff_up', 'RELPOS_diff_up','GR_diff_down', 'ILD_log10_diff_down', 'DeltaPHI_diff_down', 'PHIND_diff_down', 'PE_diff_down', 'NM_M_diff_down', 'RELPOS_diff_down','GR', 'ILD_log10', 'DeltaPHI', 'PHIND', 'PE', 'NM_M', 'RELPOS']
feature_names2 = ['GR', 'ILD_log10', 'DeltaPHI', 'PHIND', 'PE', 'NM_M', 'RELPOS']
feature_names3 = ['GR', 'ILD_log10', 'DeltaPHI', 'PHIND', 'PE', 'NM_M', 'RELPOS','GR_diff_up', 'ILD_log10_diff_up', 'DeltaPHI_diff_up', 'PHIND_diff_up', 'PE_diff_up', 'NM_M_diff_up', 'RELPOS_diff_up']
facies_names = ['SS', 'CSiS', 'FSiS', 'SiSh', 'MS', 'WS', 'D', 'PS', 'BS']
facies_colors = ['#F4D03F', '#F5B041','#DC7633','#6E2C00', '#1B4F72','#2E86C1', '#AED6F1', '#A569BD', '#196F3D']

In [4]:
[set(data['Facies'][data['Well Name'] == well]) for well in set(data['Well Name'])]


[{2, 3, 4, 5, 6, 7, 8},
 {2, 3, 4, 5, 6, 7, 8, 9},
 {1, 2, 3, 4, 5, 6, 7, 8},
 {9},
 {1, 2, 3, 4, 5, 6, 7, 8},
 {2, 3, 4, 5, 6, 7, 8, 9},
 {1, 2, 3, 4, 5, 6, 7, 8, 9},
 {1, 2, 3, 4, 5, 6, 7, 8, 9},
 {2, 3, 4, 5, 6, 7, 8, 9},
 {1, 2, 3, 4, 5, 6, 7, 8}]

In [5]:
[(face,len(data[data['Facies'] == face])) for face in set(data['Facies'])]

[(1, 268),
 (2, 940),
 (3, 780),
 (4, 271),
 (5, 296),
 (6, 582),
 (7, 141),
 (8, 686),
 (9, 185)]

In [6]:
def find_diff(row, well):
    if len(prev_depth_features[well]) == 0:
        prev_depth_features[well] = row.values[4:]
        return
    diff = row.values[4:] - prev_depth_features[well]
    prev_depth_features[well] = row.values[4:]
    return diff
data_well = dict()
data_well_inverse = dict()
prev_depth_features = dict()
new_data = pd.DataFrame()
prev_class= dict()
data_save = pd.DataFrame()
for well in set(data['Well Name']):
    prev_depth_features[well] = []
    prev_class[well] = []
    data_well[well] = data[data['Well Name'] == well]
    data_well[well] = data_well[well].sort_values(by=['Depth'])
    data_save = data_well[well].iloc[::-1]
    data_well[well]['diff_up'] = data_well[well].apply(lambda row: find_diff(row, well), axis=1)
    prev_depth_features[well] = []
    prev_class[well] = []
    #data_save = data_save.apply(lambda row: find_diff(row, well), axis=1)
    #data_well[well]['diff_down'] = data_save.iloc[::-1]
    data_well[well] = data_well[well].dropna()
    data_well[well]['GR_diff_up'] = data_well[well].apply(lambda row: row['diff_up'][0], axis=1)
    data_well[well]['ILD_log10_diff_up'] = data_well[well].apply(lambda row: row['diff_up'][1], axis=1)
    data_well[well]['DeltaPHI_diff_up'] = data_well[well].apply(lambda row: row['diff_up'][2], axis=1)
    data_well[well]['PHIND_diff_up'] = data_well[well].apply(lambda row: row['diff_up'][3], axis=1)
    data_well[well]['PE_diff_up'] = data_well[well].apply(lambda row: row['diff_up'][4], axis=1)
    data_well[well]['NM_M_diff_up'] = data_well[well].apply(lambda row: row['diff_up'][5], axis=1)
    data_well[well]['RELPOS_diff_up'] = data_well[well].apply(lambda row: row['diff_up'][6], axis=1)
    #data_well[well]['GR_diff_down'] = data_well[well].apply(lambda row: row['diff_down'][0], axis=1)
    #data_well[well]['ILD_log10_diff_down'] = data_well[well].apply(lambda row: row['diff_down'][1], axis=1)
    #data_well[well]['DeltaPHI_diff_down'] = data_well[well].apply(lambda row: row['diff_down'][2], axis=1)
    #data_well[well]['PHIND_diff_down'] = data_well[well].apply(lambda row: row['diff_down'][3], axis=1)
    #data_well[well]['PE_diff_down'] = data_well[well].apply(lambda row: row['diff_down'][4], axis=1)
    #data_well[well]['NM_M_diff_down'] = data_well[well].apply(lambda row: row['diff_down'][5], axis=1)
    #data_well[well]['RELPOS_diff_down'] = data_well[well].apply(lambda row: row['diff_down'][6], axis=1)
    new_data = pd.concat([new_data, data_well[well]])
    new_data = new_data.drop(['diff_up'], axis=1)
    #new_data = new_data.drop(['diff_down'], axis=1)

In [7]:
set(new_data['Well Name'])

{'ALEXANDER D',
 'CHURCHMAN BIBLE',
 'CROSS H CATTLE',
 'KIMZEY A',
 'LUKE G U',
 'NEWBY',
 'NOLAN',
 'Recruit F9',
 'SHANKLE',
 'SHRIMPLIN'}

In [8]:
def augment_features_window(X, N_neig):
    N_row = X.shape[0]
    N_feat = X.shape[1]
    X = np.vstack((np.zeros((N_neig, N_feat)),np.zeros((N_neig, N_feat)), X, np.zeros((N_neig, N_feat)),np.zeros((N_neig, N_feat))))
    X_aug = np.zeros((N_row, N_feat*(4*N_neig+1)))
    for r in np.arange(N_row) + N_neig:
        this_row = []
        for c in np.arange(-N_neig,N_neig+1):
            this_row = np.hstack((this_row, X[r+c]))
            if c != 0:
                this_row = np.hstack((this_row, (X[r] + X[r+c])/2))
        #print(len(this_row))
        X_aug[r-N_neig] = this_row

    return X_aug

def augment_features_gradient(X, depth):
    d_diff = np.diff(depth).reshape((-1, 1))
    d_diff[d_diff==0] = 0.001
    X_diff = np.diff(X, axis=0)
    X_grad = X_diff / d_diff
    X_grad = np.concatenate((X_grad, np.zeros((1, X_grad.shape[1]))))
    
    return X_grad

def augment_features(X, well, depth, N_neig=1):
    X_aug = np.zeros((X.shape[0], X.shape[1]*(4*N_neig+1)))
    for w in np.unique(well):
        w_idx = np.where(well == w)[0]
        X_aug_win = augment_features_window(X[w_idx, :], N_neig)
        #print(X_aug_win)
        #X_aug_grad = augment_features_gradient(X[w_idx, :], depth[w_idx])
        #print(X_aug_grad)
        X_aug[w_idx, :] = X_aug_win
        #X_aug[w_idx, :] = np.concatenate((X_aug_win, X_aug_grad), axis=1)
        
    return X_aug


In [9]:
def accuracy(y_predict, y_test):
    truth = [y_predict[index] for index in range(len(y_predict)) if y_predict[index] == y_test[index]]
    return len(truth)/len(y_predict)

In [10]:
import xgboost as xgb
from xgboost.sklearn import  XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import f1_score
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
def classification(clf):
    acc = 0
    clf.fit(X_train_robust_norm , y_train)
    y_predict = clf.predict(X_test_robust_norm)
    acc += f1_score(y_test, y_predict, average='micro')
    print('With augm',acc)
    return acc

Using TensorFlow backend.


In [11]:
def X_to_float(X):
    for row in range(len(X)):
        for column in range(len(X[0])):
            X[row, column] = float(X[row, column])
    return X

In [12]:
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
global_acc = 0
global_acc_f1 = 0
global_acc3 = 0
test = dict()
train = dict()
index = 0
for well in set(data['Well Name']):

    test[well] = new_data[new_data['Well Name'] == well]
    train[well] = new_data[new_data['Well Name'] != well]
    X_train = train[well][feature_names3].values 
    y_train = train[well]['Facies'].values 
    X_test = test[well][feature_names3].values 
    y_test = test[well]['Facies'].values 
    well_train = train[well]['Well Name'].values
    well_test = test[well]['Well Name'].values
    depth_train = train[well]['Depth'].values
    depth_test = test[well]['Depth'].values    

    X_aug_train = augment_features(X_train,well_train,depth_train)
    X_aug_test = augment_features(X_test,well_test,depth_test)

    robust = preprocessing.RobustScaler(quantile_range=(25.0, 75.0)).fit(X_aug_train)
    X_train_robust = robust.transform(X_aug_train)
    X_test_robust = robust.transform(X_aug_test)

    scaler = StandardScaler().fit(X_train_robust)
    X_train_robust_norm = scaler.transform(X_train_robust)
    X_test_robust_norm = scaler.transform(X_test_robust)

    print(well)
    clf_xgb = XGBClassifier(max_depth=3, learning_rate=0.12, n_estimators=150)
    global_acc += classification(clf_xgb)
    
print(global_acc/10)

LUKE G U
With augm 0.6369565217391304
SHRIMPLIN
With augm 0.6404255319148936
CROSS H CATTLE
With augm 0.464
Recruit F9
With augm 0.7721518987341772
SHANKLE
With augm 0.5825892857142857
NEWBY
With augm 0.5844155844155844
CHURCHMAN BIBLE
With augm 0.5955334987593052
KIMZEY A
With augm 0.5639269406392694
ALEXANDER D
With augm 0.6236559139784946
NOLAN
With augm 0.5217391304347826
0.5985394306329923


In [13]:
clf_xgb.feature_importances_

array([0.02072344, 0.01243406, 0.01733233, 0.022105  , 0.01582517,
       0.0001256 , 0.01620196, 0.01394122, 0.01645315, 0.01017332,
       0.00941974, 0.0148204 , 0.        , 0.02110023, 0.01130369,
       0.00778699, 0.01029892, 0.01055011, 0.01657875, 0.00640543,
       0.01406682, 0.01155489, 0.01708114, 0.00879176, 0.00904295,
       0.01029892, 0.        , 0.02298418, 0.01004773, 0.01029892,
       0.01243406, 0.0109269 , 0.01130369, 0.        , 0.00879176,
       0.0074102 , 0.00678222, 0.00954534, 0.01017332, 0.00615423,
       0.        , 0.0145692 , 0.04295403, 0.03114795, 0.02323537,
       0.02687767, 0.02147702, 0.0404421 , 0.03541823, 0.0148204 ,
       0.02072344, 0.01369003, 0.01155489, 0.00891736, 0.        ,
       0.02273298, 0.01494599, 0.01230847, 0.02172821, 0.0146948 ,
       0.01796031, 0.01394122, 0.0146948 , 0.01494599, 0.01406682,
       0.01444361, 0.01343883, 0.00854057, 0.00138156, 0.04307963],
      dtype=float32)

In [ ]:
clf_xgb = XGBClassifier(learning_rate=0.12, max_depth=3,n_estimators=150)
classification(clf_xgb)

In [37]:
clf_log = LogisticRegression(multi_class='ovr')
classification(clf_log)


With augm 0.2911392405063291


0.2911392405063291

In [38]:
clf_per = MLPClassifier(hidden_layer_sizes=(100,100,100))
classification(clf_per)

With augm 0.7468354430379747


0.7468354430379747

In [39]:
from keras.utils import to_categorical
y_train_tocat = to_categorical(y_train, 10)
y_test_tocat = to_categorical(y_test, 10)
model = Sequential()
model.add(Dense(100, input_dim=7, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_robust_norm, y_train_tocat, validation_split=0.3, epochs=200, batch_size=10)
model.evaluate(X_test_robust_norm, y_test_tocat)

ValueError: Error when checking input: expected dense_1_input to have shape (7,) but got array with shape (70,)

In [ ]:
import pandas as pd
df = pd.DataFrame()
df['y1'] = [1, 2]
df['y2'] = [2, 3]
df

In [ ]:
import numpy as np
def add_std(y1, y2):
    return np.std([y1, y2]), np.mean([y1, y2]), min([y1, y2])
df['Result'] = df.apply(lambda row: add_std(row['y1'], row['y2']), axis=1)

In [ ]:
df['std'] = df.apply(lambda row: row['Result'][0], axis = 1)
df['mean'] = df.apply(lambda row: row['Result'][1], axis = 1)
df['min'] = df.apply(lambda row: row['Result'][2], axis = 1)

In [ ]:
df

In [ ]:
df = df.drop(['Result'], axis=1)

In [59]:
df

,y1,y2,std,mean,min
0,1,2,0.5,1.5,1.0
1,2,3,0.5,2.5,2.0


In [ ]:
acc= 0
    for iter in range(10):
        clf.fit(X_train, y_train)
        y_predict = clf.predict(X_test)
        acc += accuracy(y_predict, y_test)
    acc /= 10
    print('without anything',acc)
    acc= 0
    for iter in range(10):
        clf.fit(X_aug_train_robust, y_aug_train)
        y_predict = clf.predict(X_aug_test_robust)
        acc += accuracy(y_predict, y_aug_test)
    acc /= 10
    print('With robust and aug',acc)
    acc = 0
    for iter in range(10):
        clf.fit(X_train_robust, y_train)
        y_predict = clf.predict(X_test_robust)
        acc += accuracy(y_predict, y_test)
    acc /= 10
    print('With robust',acc)
    acc= 0
    for iter in range(10):
        clf.fit(X_aug_train_robust_norm, y_aug_train)
        y_predict = clf.predict(X_aug_test_robust_norm)
        acc += accuracy(y_predict, y_aug_test)
    acc /= 10
    print('With robust and aug and norm',acc)
    acc = 0
    for iter in range(10):
        clf.fit(X_train_robust_norm, y_train)
        y_predict = clf.predict(X_test_robust_norm)
        acc += accuracy(y_predict, y_test)
    acc /= 10
    print('With robust and norm',acc)

In [ ]:
robust = preprocessing.RobustScaler(quantile_range=(25.0, 75.0)).fit(X_aug_train)
    X_aug_train_robust = robust.transform(X_aug_train)
    X_aug_test_robust = robust.transform(X_aug_test)

    robust = preprocessing.RobustScaler(quantile_range=(25.0, 75.0)).fit(X_train)
    X_train_robust = robust.transform(X_train)
    X_test_robust = robust.transform(X_test)

    scaler = StandardScaler().fit(X_aug_train_robust)
    X_aug_train_robust_norm = scaler.transform(X_aug_train_robust)
    X_aug_test_robust_norm = scaler.transform(X_aug_test_robust)

    scaler = preprocessing.RobustScaler(quantile_range=(25.0, 75.0)).fit(X_train_robust)
    X_train_robust_norm = scaler.transform(X_train_robust)
    X_test_robust_norm = scaler.transform(X_test_robust)